In [7]:
#import module

#python defalut module
import time
import re
import os
from datetime import datetime,timedelta
from IPython.display import Image

import pandas as pd
import numpy as np

import urllib3
from bs4 import BeautifulSoup
import requests

import selenium

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

import warnings
warnings.filterwarnings(action='ignore')

### 현대 트랜시스 뉴스 크롤링  
> 네이버 뉴스 통합검색으로 진행
>
> 통합검색시 page, start date, end date 설정
>
> 네이버 기사 제목 + URL + 기사검색시 요약본 3가지를 추출
>
> 네이버 뉴스란 클릭하면 네이버 뉴스로만 들어가짐 해당 기사 긁어오기

날짜 형식 yyyy.mm.dd로 입력? 
  
input data를 설정해야함(시작 날짜, 종료날짜, 검색어) 3개로 선정?

### 검색어 Input

검색어의 경우 기존의 재경본부에서 진행하는 현대차그룹, 현대자동차, 기아자동차, 현대트랜시스 네 가지의 검색어를 동일하게 진행하였습니다.
검색어를 선정해서 크롤링을 진행할 수 있게 작성하였고, 다른 검색어가 들어갈 경우 ERROR 코드로  
검색어 input data 가 입력값을 벗어났습니다. 다시 입력해주세요.문구가 뜨게 작성하였습니다

In [8]:
category = []
defalut_category = ['현대차그룹' ,'현대자동차', '기아자동차' , '현대트랜시스'] # 검색어 범주 지정
def select_keyword():
    while True:
        search_keywords = input("""
        크롤링 할 수 있는 검색어는 다음과 같습니다.
        ------------------------------------------------------------
        '현대차그룹' , '현대자동차', '기아자동차' , '현대트랜시스'
        ------------------------------------------------------------
        크롤링 할 분야를 최소 한가지 이상 선택하여 입력해 주세요. 띄어쓰기로 구분합니다.
        입력:""")

        input_keyword = search_keywords.split()
        is_keyword = True
        for input_data in input_keyword:
            if not input_data in defalut_category:
                print('검색어' +input_data + " 가 입력값을 벗어났습니다. 다시 입력해주세요.")
                is_keyword = False
                break
        if is_keyword is True:
            break
            
    category.extend(input_keyword)
    return input_keyword

In [9]:
select_keyword()


        크롤링 할 수 있는 검색어는 다음과 같습니다.
        ------------------------------------------------------------
        '현대차그룹' , '현대자동차', '기아자동차' , '현대트랜시스'
        ------------------------------------------------------------
        크롤링 할 분야를 최소 한가지 이상 선택하여 입력해 주세요. 띄어쓰기로 구분합니다.
        입력:현대차그룹 현대자동차 현대트랜시스


['현대차그룹', '현대자동차', '현대트랜시스']

1일 검색어 기준

In [4]:
title_list =[] # 제목을 저장하기 위한 list
url_list = [] # URL을 저장하기 위한 list
summary_list = [] # 날짜를 저장하기 위한 list
date_list = []

def Crawl(category):
    keywords = select_keyword()
    for keyword in keywords:
        page = f'https://search.naver.com/search.naver?where=news&query={keywords}&sm=tab_opt&sort=0&photo=0&field=0&reporter_article=&pd=4&ds=&de=&docid=&nso=so%3Ar%2Cp%3A1d%2Ca%3Aall&mynews=0&refresh_start=0&related=0'
        res = requests.get(page, headers={'User-Agent':
                                  ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                                   "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36")},
                   verify = False) 
        news_html = BeautifulSoup(res.text,'html.parser')

        content = news_html.select(".group_news") # 기사 본문 저장되어 있는 html주소 복사


        news_area = content[0].select('.news_area') # 기사가 저장되어 있는 링크 추출

        for area in range(len(news_area)) :
            title_content = news_area[area].select('a')

            # title content 안의 attr은 dictionary 형태기때문에 has_key함수 바탕으로 title여부를 확인하여 리스트를 뽑아낸다
            for i in title_content : 
                if i.has_key('title') == True :
                    title_list.append(i.attrs['title'])
                else : continue

            date_content = news_area[area].find_all('span',{'class':'info'})
            news_date = date_content[0].get_text()

        url_content = news_area[area].select('a')
        for i in url_content :
            url_list.append(i.attrs['href'])

In [4]:
title_list =[] # 제목을 저장하기 위한 list
url_list = [] # URL을 저장하기 위한 list
summary_list = [] # 날짜를 저장하기 위한 list

def Crawl(category):
    keywords = select_keyword()
    for keyword in keywords:
        page = f'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={keyword}'
        res = requests.get(page, headers={'User-Agent':
                                  ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                                   "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36")},
                   verify = False) 
        news_html = BeautifulSoup(res.text,'html.parser')
        
        content = news_html.select("#group_news") # 기사 본문 저장되어 있는 html주소 복사


        title_content = content[0].select('dt > a') # 기사가 저장되어 있는 링크 추출

        for i in title_content :
            title_list.append(i.attrs['title'])


        url_content = content[0].select('dt > a')
        for i in url_content :
            url_list.append(i.attrs['href'])


        summary_content = content[0].select('dl > dd')
        for k in summary_content :
            if str(k.attrs) == "{'class': ['txt_inline']}" :
                continue

            for target in k.find_all('span'): # 기사 본문에 추가로 저장되어 있는 각 class들 제거
                target.extract()

            for target in k.find_all('ul'): # 기사 본문에 추가로 저장되어 있는 각 class들 제거
                target.extract()

            for target in k.find_all('div'): # 기사 본문에 추가로 저장되어 있는 각 class들 제거
                target.extract()

            k = k.get_text().strip()

            if not k  :
                continue
            summary_list.append(k)

In [5]:
Crawl(category)


        크롤링 할 수 있는 검색어는 다음과 같습니다.
        ------------------------------------------------------------
        '현대차그룹' , '현대자동차', '기아자동차' , '현대트랜시스'
        ------------------------------------------------------------
        크롤링 할 분야를 최소 한가지 이상 선택하여 입력해 주세요. 띄어쓰기로 구분합니다.
        입력:현대트랜시스 현대차그룹


IndexError: list index out of range

In [42]:
def date_preprocessing(date) :
    
    date = date.get_text() #날짜 추출

    if '시간' in date : # 날짜가 n시간 전으로 표기될 경우 문자열 처리
        date_find = re.findall(r'\d+', date) 
        final_date = str(datetime.now() - timedelta(hours=int(date_find[0])))
        date_list.append(final_date)

    elif '분' in date : # 날짜가 n분 전으로 표기될 경우 문자열 처리
        date_find = re.findall(r'\d+', date)
        final_date = str(datetime.now() - timedelta(minutes=int(date_find[0])))
        date_list.append(final_date)

    elif '일' in date: # 날짜가 n일 전으로 표기될 경우 문자열 처리
        date_find = re.findall(r'\d+', date)
        final_date = str(datetime.now() - timedelta(days=int(date_find[0])))
        date_list.append(final_date)    

    elif bool(re.search(r'\d{3,4}.\d{1,2}.\d{2}',date)) == True : # yyyy.mm.dd로 표기될 경우 처리
        final_date = date.replace('.',"-")
        date_list.append(final_date)

In [34]:
title_list =[] # 제목을 저장하기 위한 list
url_list = [] # URL을 저장하기 위한 list
summary_list = [] # 날짜를 저장하기 위한 list
date_list = []

page = f'https://search.naver.com/search.naver?where=news&query=현대차그룹'
res = requests.get(page, headers={'User-Agent':
                          ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                           "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36")},
           verify = False) 
news_html = BeautifulSoup(res.text,'html.parser')

content = news_html.select(".group_news") # 기사 본문 저장되어 있는 html주소 복사


news_area = content[0].select('.news_area') # 기사가 저장되어 있는 링크 추출

for area in range(len(news_area)) :
    title_content = news_area[area].select('a')

    # title content 안의 attr은 dictionary 형태기때문에 has_key함수 바탕으로 title여부를 확인하여 리스트를 뽑아낸다
    for i in title_content : 
        if i.has_key('title') == True :
            title_list.append(i.attrs['title'])
        else : continue

    date_content = news_area[area].find_all('span',{'class':'info'})
    news_date = date_content[0].get_text()

url_content = news_area[area].select('a')
for i in url_content :
    url_list.append(i.attrs['href'])


# summary_content = content[0].select('dl > dd')
# for k in summary_content :
#     if str(k.attrs) == "{'class': ['txt_inline']}" :
#         continue

#     for target in k.find_all('span'): # 기사 본문에 추가로 저장되어 있는 각 class들 제거
#         target.extract()

#     for target in k.find_all('ul'): # 기사 본문에 추가로 저장되어 있는 각 class들 제거
#         target.extract()

#     for target in k.find_all('div'): # 기사 본문에 추가로 저장되어 있는 각 class들 제거
#         target.extract()

#     k = k.get_text().strip()

#     if not k  :
#         continue
#     summary_list.append(k)

In [35]:
title_list

['정의선 현대차 회장, 이건희 회장 영결식에도 달려왔다(종합)']

In [41]:
date_content = news_area[0].find_all('span',{'class':'info'})
date_content[0].get_text()

'20시간 전'

In [36]:
title_content

[<a class="info press" href="https://www.yna.co.kr/" onclick="return goOtherCR(this, 'a=nws*h.prof&amp;r=1&amp;i=880000D8_000000000000000011977771&amp;g=001.0011977771&amp;u='+urlencode(this.href));" target="_blank"><span class="thumb_box"></span>연합뉴스<i class="spnew ico_pick">언론사 선정</i></a>,
 <a class="info" href="https://news.naver.com/main/read.nhn?mode=LSD&amp;mid=sec&amp;sid1=101&amp;oid=001&amp;aid=0011977771" onclick="return goOtherCR(this, 'a=nws*h.nav&amp;r=1&amp;i=880000D8_000000000000000011977771&amp;u='+urlencode(this.href));" target="_blank">네이버뉴스</a>,
 <a class="news_tit" href="http://yna.kr/AKR20201028070351003?did=1195m" onclick="return goOtherCR(this, 'a=nws*h.tit&amp;r=1&amp;i=880000D8_000000000000000011977771&amp;g=001.0011977771&amp;u='+urlencode(this.href));" target="_blank" title="정의선 현대차 회장, 이건희 회장 영결식에도 달려왔다(종합)">정의선 <mark>현대차</mark> 회장, 이건희 회장 영결식에도 달려왔다(종합)</a>,
 <a class="api_txt_lines dsc_txt_wrap" href="http://yna.kr/AKR20201028070351003?did=1195m" onclick="

In [ ]:
title

## TEST

In [73]:
final_df

,title,article
0,"`자율주행 우체국이 집앞까지` 비대면으로 접수하고, 배달받는다(종합)","<div class=""_article_body_contents"" id=""articl..."
1,"장대환 매경 회장 ""MBN 불법 행위, 나는 몰랐다""","<div class=""_article_body_contents"" id=""articl..."
2,"文 ""나의아저씨·슈스케 제작한 케이블TV 뿌듯""…25주년 시상식 개최","<div class=""_article_body_contents"" id=""articl..."
3,임플란트 표면 감염균 부착 막아줄 코팅 기술 개발,"<div class=""_article_body_contents"" id=""articl..."
4,게임업계 “이젠 PC·콘솔보다 모바일”,"<div class=""_article_body_contents"" id=""articl..."
5,"맵퍼스-도로공사, “사고포착알리미 시행 후 돌발상황 인지 시간 단축”","<div class=""node_body cb"">맵퍼스는 아틀란 내비게이션 앱의 실시..."
6,"한국IBM, 뉴칼라 인재 육성 위한 무료 오픈 교육 플랫폼 발표","<div class=""node_body cb"">IBM이 디지털 뉴딜에 맞춤화된 뉴칼..."
7,"2020 하반기 기업 채용, 오픈소스 전문가에 눈 뜨다","<div class=""node_body cb"">모든 이들에게 험난한 한 해였다. 그..."
8,IDG 블로그 | 클라우드옵스가 잘 안되는 이유,"<div class=""node_body cb"">클라우드 운영, 즉 클라우드옵스(Cl..."
9,"마인즈랩 유태준 대표 ""AI 분야 넷플릭스 되겠다""","<figure class=""view_cont_img_jpg""><span class=..."


In [66]:
category = []
defalut_category = ['현대차그룹' ,'현대자동차', '기아자동차' , '현대트랜시스']
def select_keyword():
    while True:
        search_keywords = input("""
        크롤링 할 수 있는 검색어는 다음과 같습니다.
        ------------------------------------------------------------
        '현대차그룹' , '현대자동차', '기아자동차' , '현대트랜시스'
        ------------------------------------------------------------
        크롤링 할 분야를 최소 한가지 이상 선택하여 입력해 주세요. 띄어쓰기로 구분합니다.
        입력:""")

        input_keyword = search_keywords.split()
        is_keyword = True
        for input_data in input_keyword:
            if not input_data in defalut_category:
                print(input_data + " 가 입력값을 벗어났습니다. 다시 입력해주세요.")
                is_keyword = False
                break
        if is_keyword is True:
            break
            
    category.extend(input_keyword)
    return input_keyword


In [80]:
final_df.drop([1,2,3,4,5,6,7,9,11,14,15,16,18,20,21],axis=0,inplace=True)

In [69]:
select_keyword()


        크롤링 할 수 있는 검색어는 다음과 같습니다.
        ------------------------------------------------------------
        '현대차그룹' , '현대자동차', '기아자동차' , '현대트랜시스'
        ------------------------------------------------------------
        크롤링 할 분야를 최소 한가지 이상 선택하여 입력해 주세요. 띄어쓰기로 구분합니다.
        입력:현대트랜시스 현대차그룹


['현대트랜시스', '현대차그룹']

In [ ]:
date = soup.find_all("span",{"class":bar})
# 날짜가 기록된 태그 저장
start_date = datetime.now() # 현재 날짜 저장

for j in date :
    date_list = j.get_text() #날짜 추출



#     if '시간' in date_list : # 날짜가 n시간 전으로 표기될 경우 문자열 처리
#         date_find = re.findall(r'\d+', date_list) 
#         final_date = str(datetime.now() - timedelta(hours=int(date_find[0])))
#         total_date.append(final_date)

#     elif '분' in date_list : # 날짜가 n분 전으로 표기될 경우 문자열 처리
#         date_find = re.findall(r'\d+', date_list)
#         final_date = str(datetime.now() - timedelta(minutes=int(date_find[0])))
#         total_date.append(final_date)

#     elif '일' in date_list: # 날짜가 n일 전으로 표기될 경우 문자열 처리
#         date_find = re.findall(r'\d+', date_list)
#         final_date = str(datetime.now() - timedelta(days=int(date_find[0])))
#         total_date.append(final_date)    

#     elif bool(re.search(r'\d{3,4}.\d{1,2}.\d{2}',date_list)) == True : # yyyy.mm.dd로 표기될 경우 처리
#         final_date = date_list.replace('.',"-")
#         total_date.append(final_date)

In [58]:
title_list =[] # 제목을 저장하기 위한 list
url_list = [] # URL을 저장하기 위한 list
summary_list = [] # 날짜를 저장하기 위한 list

def Crawl(data):
    keywords = select_keyword()
    for keyword in keywords:
        page = f'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={keyword}'
        res = requests.get(page, headers={'User-Agent':
                                  ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                                   "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36")},
                   verify = False) 
        news_html = BeautifulSoup(res.text,'html.parser')
        
        content = news_html.select(".type01") # 기사 본문 저장되어 있는 html주소 복사


        title_content = content[0].select('dt > a') # 기사가 저장되어 있는 링크 추출

        for i in title_content :
            title_list.append(i.attrs['title'])


        url_content = content[0].select('dt > a')
        for i in url_content :
            url_list.append(i.attrs['href'])


        summary_content = content[0].select('dl > dd')
        for k in summary_content :
            if str(k.attrs) == "{'class': ['txt_inline']}" :
                continue

            for target in k.find_all('span'): # 기사 본문에 추가로 저장되어 있는 각 class들 제거
                target.extract()

            for target in k.find_all('ul'): # 기사 본문에 추가로 저장되어 있는 각 class들 제거
                target.extract()

            for target in k.find_all('div'): # 기사 본문에 추가로 저장되어 있는 각 class들 제거
                target.extract()

            k = k.get_text().strip()

            if not k  :
                continue
            summary_list.append(k)

In [59]:
Crawl()

TypeError: Crawl() missing 1 required positional argument: 'data'

In [50]:
title_list =[] # 제목을 저장하기 위한 list
url_list = [] # URL을 저장하기 위한 list
date_list = [] # 날짜를 저장하기 위한 list
summary_list = []

test = '현대트랜시스'
page = f'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={test}'
res = requests.get(page, headers={'User-Agent':
                          ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                           "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36")},
           verify = False) 
news_html = BeautifulSoup(res.text,'html.parser')

content = news_html.select(".type01") # 기사 본문 저장되어 있는 html주소 복사


title_content = content[0].select('dt > a') # 기사가 저장되어 있는 링크 추출

for i in title_content :
    title_list.append(i.attrs['title'])
    

url_content = content[0].select('dt > a')
for i in url_content :
    url_list.append(i.attrs['href'])
    
    
summary_content = content[0].select('dl > dd')
for k in summary_content :
    if str(k.attrs) == "{'class': ['txt_inline']}" :
        continue
        
    for target in k.find_all('span'): # 기사 본문에 추가로 저장되어 있는 각 class들 제거
        target.extract()

    for target in k.find_all('ul'): # 기사 본문에 추가로 저장되어 있는 각 class들 제거
        target.extract()
        
    for target in k.find_all('div'): # 기사 본문에 추가로 저장되어 있는 각 class들 제거
        target.extract()
    
    k = k.get_text().strip()
    
    if not k  :
        continue
    summary_list.append(k)

In [51]:
print(len(title_list))
print(len(url_list))
print(len(summary_list))

10
10
10


In [52]:
df = pd.DataFrame({'title': title_list,'url':url_list,'summary':summary_list}) 


In [53]:
df

,title,url,summary
0,"현대차, 친환경 사회공헌 캠페인 '언택트 펀 레이스' 실시",http://www.etnews.com/20201019000011,"현대트랜시스로부터 제공받은 자동차 시트 폐자재를 재활용하고, 글로벌 재활용 컨설팅 ..."
1,"정의선, 현대차그룹 회장 취임 고사하다 결국 수락한 이유",https://www.hankyung.com/economy/article/20201...,"실제 HMGICS의 경우 현대차와 기아차는 물론 현대모비스, 현대글로비스, 현대오토..."
2,현대차 정의선체제 `3세 경영` 가속화,http://www.dt.co.kr/contents.html?article_no=2...,"이번 사업에는 현대·기아차를 비롯해 현대모비스, 현대글로비스, 현대오토에버, 현대위..."
3,현대차그룹 '싱가포르 글로벌 혁신센터' 기공,http://news.kbiz.or.kr/news/articleView.html?i...,"현대·기아자동차 이외에도 현대모비스, 현대글로비스, 현대오토에버, 현대위아, 현대로..."
4,"[현대차 뉴리더십] 아킬레스건, 지배구조 개편 남았다",https://biz.chosun.com/site/data/html_dir/2020...,한편 계열사 개편 과정에서 현대엔지니어링과 현대트랜시스 등 일부 계열사의 사업 영역...
5,"현대차의 혁신 실험, 싱가포르 글로벌 혁신센터 기공식 개최",http://it.chosun.com/site/data/html_dir/2020/1...,"현대·기아자동차 이외에도 현대모비스, 현대글로비스, 현대오토에버, 현대위아, 현대로..."
6,"'코로나 넘어 리콜' 현대차, 또 실력 나올까",http://news.bizwatch.co.kr/article/industry/20...,"한신평이 등급을 부여한 현대트랜시스, 현대위아, 현대케피코 3사는 올해 2분기 합산..."
7,"현대차그룹, 싱가포르 글로벌 혁신센터 기공식 개최",http://kr.aving.net/news/view.php?articleId=15...,"현대·기아자동차 이외에도 현대모비스, 현대글로비스, 현대오토에버, 현대위아, 현대로..."
8,"현대자동차, 서울어린이대공원서 '언택트 펀 레이스'",http://www.safetimes.co.kr/news/articleView.ht...,현대트랜시스로부터 받은 자동차 시트 폐자재를 재활용해 글로벌 재활용 컨설팅 기업 테...
9,"현대차그룹, ‘미래차 시험 무대’로 왜 동남아를 선택했나",https://www.dnews.co.kr/uhtml/view.jsp?idxno=2...,"현대모비스, 현대글로비스, 현대오토에버, 현대위아, 현대로템, 현대트랜시스 등 그룹..."


In [18]:
summary_content = content[0].select('dl > dd')
summary_content

for i in summary_content :
    if str(i.attrs) == "{'class': ['txt_inline']}" :
        continue
    print(i.get_text())


현대트랜시스로부터 제공받은 자동차 시트 폐자재를 재활용하고, 글로벌 재활용 컨설팅 기업 테라사이클이 재활용 연구개발(R&D) 및 트랙 제작을 담당한 러닝 트랙은 서울어린이대공원에 설치돼 어린이들이 마음껏...
    현대차, 2020 롱기스트 런 파이널 이벤트 '언택트 펀 레이스' 실시  에이빙뉴스  2020.10.19.     현대차, '2020 롱기스트 런' 서울어린이대공원서 개최  이데일리  2020.10.19. 네이버뉴스     현대차, 2020 롱기스트 런 파이널 참가자 모집… 달리면서 환경 보호 실천  동아일보  2020.10.19. 네이버뉴스     현대차, 2020 롱기스트 런 파이널 이벤트 ‘언택트 펀 레이스’  e대한경제  2020.10.19.     관련뉴스 26건 전체보기  
실제 HMGICS의 경우 현대차와 기아차는 물론 현대모비스, 현대글로비스, 현대오토에버, 현대위아, 현대로템, 현대트랜시스 등 현대차그룹 그룹사들이 대거 참여한다. 이들의 역량을 결집시키려면 책임경영이 가능한...
    현대차그룹 ‘3세 경영’ 본격화…정의선 시대 포문  스카이데일리  2020.10.14.    
이번 사업에는 현대·기아차를 비롯해 현대모비스, 현대글로비스, 현대오토에버, 현대위아, 현대로템, 현대트랜시스 등 그룹사들이 대거 참여한다. 한편 이날 기공식에는 싱가포르에서는 리센룽 총리, 안영집...
    현대차, 싱가포르에 모빌리티 혁신기지  디지털타임스  2020.10.13. 네이버뉴스     정의선 "인간 중심 밸류체인 혁신"…현대차그룹, 싱가포르 글로벌 혁신센터 건...  디지털타임스  2020.10.13. 네이버뉴스    
현대·기아자동차 이외에도 현대모비스, 현대글로비스, 현대오토에버, 현대위아, 현대로템, 현대트랜시스 등 현대자동차그룹 그룹사들이 대거 HMGICS에 참여한다. 베 스완 진 싱가포르 경제개발청(EDB)장은...
    현대차 ‘미래 모빌리티 실험실’ 싱가포르 글로벌혁신센터 착공  한겨레  2020.10.13. 네이버뉴스   

In [30]:
url_list

['https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=103&oid=030&aid=0002907949',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=015&aid=0004431285',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=029&aid=0002629793',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=103&oid=366&aid=0000603102']

In [27]:
title_list

["현대차, 친환경 사회공헌 캠페인 '언택트 펀 레이스' 실시",
 '정의선, 현대차그룹 회장 취임 고사하다 결국 수락한 이유',
 '현대차 정의선체제 `3세 경영` 가속화',
 "현대차그룹 '싱가포르 글로벌 혁신센터' 기공",
 '[현대차 뉴리더십] 아킬레스건, 지배구조 개편 남았다',
 '현대차의 혁신 실험, 싱가포르 글로벌 혁신센터 기공식 개최',
 "'코로나 넘어 리콜' 현대차, 또 실력 나올까",
 '현대차그룹, 싱가포르 글로벌 혁신센터 기공식 개최',
 "현대자동차, 서울어린이대공원서 '언택트 펀 레이스'",
 '현대차그룹, ‘미래차 시험 무대’로 왜 동남아를 선택했나']

In [17]:
for i in title_content:
    print()

[<a class="link_help" href="https://help.naver.com/support/alias/search/word/word_1.naver" onclick="return goOtherCR(this, 'a=rsk_top.guide&amp;r=&amp;i=&amp;u='+urlencode(urlexpand(this.href)));" target="_blank"><i class="sprenew api_ico_help">도움말</i></a>,
 <a class="_sp_each_title" href="http://www.etnews.com/20201019000011" onclick="return goOtherCR(this, 'a=nws*h.tit&amp;r=1&amp;i=880000EA_000000000000000002907949&amp;g=030.0002907949&amp;u='+urlencode(this.href));" target="_blank" title="현대차, 친환경 사회공헌 캠페인 '언택트 펀 레이스' 실시"><strong class="hl">현대</strong>차, 친환경 사회공헌 캠페인 '언택트 펀 레이스' 실시</a>,
 <a class="_sp_each_title" href="https://www.hankyung.com/economy/article/202010145803g" onclick="return goOtherCR(this, 'a=nws*h.tit&amp;r=6&amp;i=88000107_000000000000000004431285&amp;g=015.0004431285&amp;u='+urlencode(this.href));" target="_blank" title="정의선, 현대차그룹 회장 취임 고사하다 결국 수락한 이유">정의선, <strong class="hl">현대</strong>차그룹 회장 취임 고사하다 결국 수락한 이유</a>,
 <a class="_sp_each_title" href="http://www.dt

In [ ]:
class input_date:
    def __init__(self, s_date, e_date , search_keyword):
        self.s_date = start_date
        self.e_date = end_date
        self.search_keyword = search_keyword
        
    def set_contact():
        start_date = int(input("날짜를 yyyy.mm.dd 형식으로 입력하세요: "))
        end_date = int(input("날짜를 yyyy.mm.dd 형식으로 입력하세요: "))
        search_keyword = input('검색어를 쉼표로 구분하여 넣으세요:').split(',')
        print(start_date, end_date, search_keyword)


    def get_days_list(self):
        days_list = []
        day = datetime.date.today()
        for i in range(0, self.days):
            day = day - datetime.timedelta(days=1)
            days_list.append(day.isoformat())
        return days_list

def today_date():
    # 'yyyy-mm-mm' 형식 날짜 반환
    now = datetime.date.today()
    return now.isoformat()

def date_preprocessing(soup, date_class) :
    date = soup.find_all("span",{"class":bar})
    # 날짜가 기록된 태그 저장
    start_date = datetime.now() # 현재 날짜 저장

    for j in date :
        date_list = j.get_text() #날짜 추출

        if '시간' in date_list : # 날짜가 n시간 전으로 표기될 경우 문자열 처리
            date_find = re.findall(r'\d+', date_list) 
            final_date = str(datetime.now() - timedelta(hours=int(date_find[0])))
            total_date.append(final_date)

        elif '분' in date_list : # 날짜가 n분 전으로 표기될 경우 문자열 처리
            date_find = re.findall(r'\d+', date_list)
            final_date = str(datetime.now() - timedelta(minutes=int(date_find[0])))
            total_date.append(final_date)

        elif '일' in date_list: # 날짜가 n일 전으로 표기될 경우 문자열 처리
            date_find = re.findall(r'\d+', date_list)
            final_date = str(datetime.now() - timedelta(days=int(date_find[0])))
            total_date.append(final_date)    

        elif bool(re.search(r'\d{3,4}.\d{1,2}.\d{2}',date_list)) == True : # yyyy.mm.dd로 표기될 경우 처리
            final_date = date_list.replace('.',"-")
            total_date.append(final_date)